# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [2]:
gdf = cudf.read_csv('./data/week2.csv', usecols = ['lat','long','infected'])

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [3]:
hospitals = cudf.read_csv('./data/hospitals.csv', usecols=['Latitude', 'Longitude'])
clinics = cudf.read_csv('./data/clinics.csv', usecols=['Latitude', 'Longitude'])
all_med = all_med = clinics.append(hospitals, ignore_index=True)

Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [4]:
all_med.dropna(inplace=True)
all_med.reset_index(drop=True, inplace=True)
all_med.head()

,Latitude,Longitude
0,51.804237,1.186376
1,51.815262,1.154707
2,51.780621,1.117907
3,53.482368,-2.885404
4,53.415630,-2.800874


## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [6]:
%%time
cupylat = cp.asarray(all_med['Latitude'])
cupylong = cp.asarray(all_med['Longitude'])


northing, easting = latlong2osgbgrid_cupy(cupylat,cupylong)
all_med['northing'] = cudf.Series(northing).astype('float32')
all_med['easting'] = cudf.Series(easting).astype('float32')

all_med.head()

CPU times: user 105 ms, sys: 8.3 ms, total: 113 ms
Wall time: 113 ms


,Latitude,Longitude,northing,easting
0,51.804237,1.186376,216584.968750,619651.68750
1,51.815262,1.154707,217715.546875,617415.93750
2,51.780621,1.117907,213755.250000,615045.25000
3,53.482368,-2.885404,398798.156250,341250.93750
4,53.415630,-2.800874,391308.062500,346776.28125


## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [7]:
knn = cuml.NearestNeighbors(n_neighbors = 1)

In [8]:
knn.fit(all_med[['northing','easting']])

NearestNeighbors()

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [9]:
infected_gdf = gdf[gdf['infected'] == 1].reset_index(drop=True)
infected_gdf.head()

,lat,long,infected
0,53.715826,-2.430079,1.0
1,53.664881,-2.425673,1.0
2,53.696765,-2.488940,1.0
3,53.696966,-2.488897,1.0
4,53.727804,-2.392959,1.0


Create `northing` and `easting` values for `infected_gdf`.

In [10]:
cupylat = cp.asarray(infected_gdf['lat'])
cupylong = cp.asarray(infected_gdf['long'])


northing, easting = latlong2osgbgrid_cupy(cupylat,cupylong)
infected_gdf['northing'] = cudf.Series(northing).astype('float32')
infected_gdf['easting'] = cudf.Series(easting).astype('float32')

infected_gdf.head()

,lat,long,infected,northing,easting
0,53.715826,-2.430079,1.0,424489.78125,371619.68750
1,53.664881,-2.425673,1.0,418820.68750,371876.50000
2,53.696765,-2.488940,1.0,422394.40625,367721.00000
3,53.696966,-2.488897,1.0,422416.81250,367723.96875
4,53.727804,-2.392959,1.0,425808.12500,374076.56250


Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [11]:
distances, indices = knn.kneighbors(infected_gdf[['northing','easting']], 1)

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [12]:
indices.head()

0    17090
1    11590
2     9508
3     9508
4     3736
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [13]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat          5.371583e+01
long        -2.430079e+00
infected    4.100745e-322
northing     4.244898e+05
easting      3.716197e+05
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [14]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

Latitude     1.000665e+28
Longitude    3.231861e+24
northing     5.507473e+05
easting      4.110586e+05
Name: 1234, dtype: float64

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [15]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}